# Day 1

## Part 1
Figure out the most calories carried by any elf.

In [64]:
; TODO stream process line by by line rather than read in the entire file into memory
(let ((lines (uiop:read-file-lines "input_day1_part1.txt"))
      ; temporary accumulator for each elfs calories
      (cal_ct 0)
      ; variable to track the maximum calories per elf
      (max_cal 0))
    (dolist (line lines)
        ; elves are demarcated by empty lines
        (if (string= "" line)
            (progn (when (> cal_ct max_cal) (setq max_cal cal_ct)) (setq cal_ct 0))
            (incf cal_ct (parse-integer line))
        )
    )
 ; return the maximum number of calories found on an elf
 max_cal)
    

68923

## Part 2

Find the sum of the calories for the top 3 elves. 

In [8]:
(let (
      (lines (uiop:read-file-lines "input_day1_part1.txt"))
      (cal_ct 0)
      (calories nil))
    (dolist (line lines)
        (if (string= "" line)
            (progn (push cal_ct calories) (setq cal_ct 0))
            (progn (incf cal_ct (parse-integer line)))
        )
    )
    (let ((sorted (sort calories #'>)))
         (+ (first sorted) (second sorted) (third sorted)))
)


200044